In [5]:
import numpy as np
import pandas as pd
import re
from sklearn.metrics import mean_squared_error, classification_report
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

In [6]:
df = pd.read_csv('smoke_detection_iot.csv')
df.head()

,Unnamed: 0,UTC,Temperature[C],Humidity[%],TVOC[ppb],eCO2[ppm],Raw H2,Raw Ethanol,Pressure[hPa],PM1.0,PM2.5,NC0.5,NC1.0,NC2.5,CNT,Fire Alarm
0,0,1654733331,20.000,57.36,0,400,12306,18520,939.735,0.0,0.0,0.0,0.0,0.0,0,0
1,1,1654733332,20.015,56.67,0,400,12345,18651,939.744,0.0,0.0,0.0,0.0,0.0,1,0
2,2,1654733333,20.029,55.96,0,400,12374,18764,939.738,0.0,0.0,0.0,0.0,0.0,2,0
3,3,1654733334,20.044,55.28,0,400,12390,18849,939.736,0.0,0.0,0.0,0.0,0.0,3,0
4,4,1654733335,20.059,54.69,0,400,12403,18921,939.744,0.0,0.0,0.0,0.0,0.0,4,0


In [7]:
df.drop(columns=['UTC', 'CNT','TVOC[ppb]' ,'Raw H2' ,'Raw Ethanol' ,'Pressure[hPa]', 'PM1.0' ,'PM2.5' ,'NC0.5' ,'NC1.0','NC2.5' ,'CNT'], inplace=True)
df.head(10)

,Unnamed: 0,Temperature[C],Humidity[%],eCO2[ppm],Fire Alarm
0,0,20.000,57.36,400,0
1,1,20.015,56.67,400,0
2,2,20.029,55.96,400,0
3,3,20.044,55.28,400,0
4,4,20.059,54.69,400,0
5,5,20.073,54.12,400,0
6,6,20.088,53.61,400,0
7,7,20.103,53.20,400,0
8,8,20.117,52.81,400,0
9,9,20.132,52.46,400,0


In [8]:
regex = re.compile(r"\[|\]|<", re.IGNORECASE)
df.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in df.columns.values]

In [9]:
Y = df['Fire Alarm']
X = df.drop(columns=['Fire Alarm'])

In [15]:
 X_train , X_test , y_train , y_test = train_test_split(X ,Y,   random_state = 42,     test_size =0.3)

In [16]:
xgb = XGBClassifier(n_jobs = -1)
xgb = xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)
mean_squared_error(y_test, y_pred)

0.0002661131513119378

In [17]:
xgb.score(X_test,y_test)

0.999733886848688

In [21]:
import pickle
with open('crop_fire.pkl', 'wb') as f:
    pickle.dump(xgb, f)